In [6]:
from Data.quantum_dataset import QuantumDataset
import torch

torch.cuda.empty_cache()
dset = QuantumDataset('all')
files = dset.get_files()


HO_gen2_0010.h5
calculated_energy: (5000, 1)
cx: (5000, 1)
cy: (5000, 1)
kx: (5000, 1)
ky: (5000, 1)
potential: (5000, 1, 256, 256)
psi: (5000, 1, 256, 256)
theoretical_energy: (5000, 1)

IW_gen2_0010.h5
calculated_energy: (5000, 1)
cx: (5000, 1)
cy: (5000, 1)
eigenvalues: (5000, 5)
potential: (5000, 1, 256, 256)
theoretical_energy: (5000, 1)
wavefunction: (5000, 1, 256, 256)
wx: (5000, 1)
wy: (5000, 1)

NG_gen2b_0000.h5
calculated_energy: (5000, 1)
eigenvalues: (5000, 5)
parameters: (5000, 10)
potential: (5000, 1, 256, 256)
wavefunction: (5000, 1, 256, 256)

RND_0011.h5
calculated_energy: (5000, 1)
eigenvalues: (5000, 5)
potential: (5000, 1, 256, 256)
wavefunction: (5000, 1, 256, 256)

RND_KE_gen2_0010.h5
calculated_energy: (5000, 1)
eigenvalues: (5000, 5)
kinetic_energy: (5000, 1)
potential: (5000, 1, 256, 256)
wavefunction: (5000, 1, 256, 256)



  0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
from torch import nn

class energy_loss(nn.Module):
    def __init__(self,alpha=1.0,gamma=0.0,dx=0.157,loss_fn=nn.L1Loss(reduction='mean')):
        super().__init__()
        self.alpha = alpha
        self.loss_fn = loss_fn
        with torch.no_grad():
            self.laplacian = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1,padding_mode='reflect',bias=False)
            kernel = (1-gamma)*torch.tensor([[0,1,0],[1,-4,1],[0,1,0]])+gamma*torch.tensor([[1/2,0,1/2],[0,-2,0],[1/2,0,1/2]])

            self.laplacian.weight[:] = kernel.reshape(1,3,3)/(dx**2)

    def forward(self,wavefunction,potential,energy):
        E = self.get_energy(wavefunction,potential)
        loss = self.loss_fn(E,energy)
        return loss*self.alpha

    def get_ke(self,wavefunction):
        return -1/2 * self.laplacian(wavefunction)

    def get_energy(self,wavefunction,potential):
        E = (self.get_ke(wavefunction) * wavefunction + potential * wavefunction**2).sum(dim=(1,2))/(wavefunction**2).sum(dim=(1,2))
        return E

In [15]:
from tqdm.notebook import tqdm
import numpy as np

dxes = np.linspace(0.15685,0.15687,1000)
'''
#dxes = [0.1568627127127127]
dxes = np.linspace(0.1568627,0.1568630,10000)
dxes = np.linspace(0.1568627591359135,0.1568627591359136,10000)
dxes = [0.15686276297629764,0.15686275]
'''

#dxes = np.linspace(0.15686272,0.156862752,11)
dxes = [0.157]
total_sums = []

for dx in tqdm(list(dxes)):
    #print(f'\ndx = {dx}:') ##dx = 0.157
    energy_loss_fn = energy_loss(alpha=1,gamma=0,dx=dx,loss_fn=nn.L1Loss(reduction='mean'))
    sums = {}
    for idx in tqdm(range(len(dset))):
        potential = dset[idx]['potential']
        wavefunction = dset[idx]['wavefunction']
        energy = dset[idx]['energy']
        potential_label = files[dset[idx]['potential_label']]
        loss = energy_loss_fn(torch.sqrt(wavefunction),potential,energy).item()
        if potential_label not in sums:
            sums[potential_label] = loss
        else:
            sums[potential_label] += loss
    l_size = (len(dset)/len(sums.keys()))
    for skey in sums:
        sums[skey] = sums[skey]/l_size
    #print(sums)

    total_sum = 0
    for potential_type in sums.keys():
        total_sum += sums[potential_type]
    total_sum = total_sum/len(sums.keys())
    #print(f'total_sum = {total_sum}')
    total_sums.append(total_sum)


min_sum = min(total_sums)
min_dx = dxes[total_sums.index(min_sum)]
print(f'Min dx = {min_dx}')
print(f' min sum = {min_sum}')

  0%|          | 0/1 [00:00<?, ?it/s]


dx = 0.15625:


  0%|          | 0/25000 [00:00<?, ?it/s]

{'HO_gen2_0010.h5': 0.003416741108894348, 'IW_gen2_0010.h5': 0.011314569348096848, 'NG_gen2b_0000.h5': 0.0007640878438949585, 'RND_0011.h5': 0.0057873205527663235, 'RND_KE_gen2_0010.h5': 0.0005332638047635556}
total_sum = 0.004363196531683207
Min dx = 0.15625
 min sum = 0.004363196531683207


# Conclusion

Gamma = 0:
Min dx = 0.15688888888888888
 min sum = 0.0034596399594843384